# **DATA3888 Project: Optiver**

In [1]:
import os
import pandas as pd
import dask.dataframe as dd

In [2]:
def load_data(directory: str) -> pd.DataFrame:

    all_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".csv"):
                all_files.append(os.path.join(root, file))

    if not all_files:
        raise FileNotFoundError("No CSV files found in the given directory.")

    df = dd.read_csv(all_files)
    return df.compute()

data_path = "./Data/individual_book_train"
df = load_data(data_path)

## **Data Exploration**

In [ ]:
df.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.000129,1.000386,0.999871,1.000643,302,615,500,400,13
1,5,1,1.000129,1.000386,0.999871,1.000643,602,515,400,500,13
2,5,2,1.000129,1.000386,0.999871,1.000643,502,515,400,500,13
3,5,3,1.000129,1.000386,0.999871,1.000643,502,515,400,500,13
4,5,4,1.000129,1.000386,0.999871,1.000643,502,515,400,600,13


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167253289 entries, 0 to 962099
Data columns (total 11 columns):
 #   Column             Dtype  
---  ------             -----  
 0   time_id            int64  
 1   seconds_in_bucket  int64  
 2   bid_price1         float64
 3   ask_price1         float64
 4   bid_price2         float64
 5   ask_price2         float64
 6   bid_size1          int64  
 7   ask_size1          int64  
 8   bid_size2          int64  
 9   ask_size2          int64  
 10  stock_id           int64  
dtypes: float64(4), int64(7)
memory usage: 15.0 GB


In [ ]:
df.describe()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
count,1.672533e+08,1.672533e+08,1.672533e+08,1.672533e+08,1.672533e+08,1.672533e+08,1.672533e+08,1.672533e+08,1.672533e+08,1.672533e+08,1.672533e+08
mean,1.602237e+04,2.969969e+02,9.997121e-01,1.000283e+00,9.995184e-01,1.000479e+00,9.285549e+02,9.233744e+02,1.181631e+03,1.146534e+03,6.271922e+01
std,9.370937e+03,1.734195e+02,3.811545e-03,3.810885e-03,3.821979e-03,3.820810e-03,5.782958e+03,5.263738e+03,7.168244e+03,6.121242e+03,3.692018e+01
min,5.000000e+00,0.000000e+00,8.807735e-01,8.876458e-01,8.806137e-01,8.898833e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,7.837000e+03,1.460000e+02,9.984497e-01,9.989405e-01,9.982569e-01,9.991112e-01,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,3.200000e+01
50%,1.584500e+04,2.960000e+02,9.998062e-01,1.000211e+00,9.996398e-01,1.000380e+00,1.610000e+02,1.610000e+02,1.590000e+02,1.610000e+02,6.200000e+01
75%,2.395800e+04,4.470000e+02,1.001055e+00,1.001535e+00,1.000888e+00,1.001728e+00,4.000000e+02,3.970000e+02,5.000000e+02,5.000000e+02,9.500000e+01
max,3.276700e+04,5.990000e+02,1.125048e+00,1.127150e+00,1.124570e+00,1.127245e+00,1.051433e+06,6.462940e+05,9.801370e+05,8.501390e+05,1.260000e+02


In [ ]:
df.isnull().sum()

time_id              0
seconds_in_bucket    0
bid_price1           0
ask_price1           0
bid_price2           0
ask_price2           0
bid_size1            0
ask_size1            0
bid_size2            0
ask_size2            0
stock_id             0
dtype: int64

## **Feature Engineering**

In [4]:
def create_features(df):
    # --- Level 1 and Level 2 Price Features ---
    # Mid prices and spreads
    df['mid_price1'] = (df['bid_price1'] + df['ask_price1']) / 2
    df['spread1'] = df['ask_price1'] - df['bid_price1']
    df['mid_price2'] = (df['bid_price2'] + df['ask_price2']) / 2
    df['spread2'] = df['ask_price2'] - df['bid_price2']
    
    # Relative spreads (spread as a fraction of mid price)
    df['relative_spread1'] = df['spread1'] / df['mid_price1']
    df['relative_spread2'] = df['spread2'] / df['mid_price2']
    
    # Price differences between levels
    df['bid_price_diff'] = df['bid_price1'] - df['bid_price2']
    df['ask_price_diff'] = df['ask_price2'] - df['ask_price1']
    df['mid_price_diff'] = df['mid_price1'] - df['mid_price2']
    
    # --- Order Size Features ---
    # Imbalance at each level (difference over sum)
    df['order_imbalance_level1'] = (df['bid_size1'] - df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    df['order_imbalance_level2'] = (df['bid_size2'] - df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    
    # Combined sizes and overall order imbalance
    df['total_bid_size'] = df['bid_size1'] + df['bid_size2']
    df['total_ask_size'] = df['ask_size1'] + df['ask_size2']
    df['total_order_imbalance'] = (df['total_bid_size'] - df['total_ask_size']) / (df['total_bid_size'] + df['total_ask_size'])
    
    # A weighted mid price for level 1 (using sizes as weights)
    df['weighted_mid_price1'] = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    
    # --- Additional Derived Features ---
    # Difference between the spreads of the two levels
    df['spread_diff'] = df['spread1'] - df['spread2']
    
    # --- Rolling (Temporal) Features ---
    # Make sure data is sorted by time; adjust window size as needed
    df = df.sort_values('time_id')
    window = 10  # Example window size
    df['rolling_mid_price1_mean'] = df['mid_price1'].rolling(window=window).mean()
    df['rolling_mid_price1_std'] = df['mid_price1'].rolling(window=window).std()
    df['rolling_spread1_mean'] = df['spread1'].rolling(window=window).mean()
    df['rolling_spread1_std'] = df['spread1'].rolling(window=window).std()
    
    return df

# Apply the feature engineering function
df_features = create_features(df)
print(df_features.info())

<class 'pandas.core.frame.DataFrame'>
Index: 167253289 entries, 0 to 962099
Data columns (total 31 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   time_id                  int64  
 1   seconds_in_bucket        int64  
 2   bid_price1               float64
 3   ask_price1               float64
 4   bid_price2               float64
 5   ask_price2               float64
 6   bid_size1                int64  
 7   ask_size1                int64  
 8   bid_size2                int64  
 9   ask_size2                int64  
 10  stock_id                 int64  
 11  mid_price1               float64
 12  spread1                  float64
 13  mid_price2               float64
 14  spread2                  float64
 15  relative_spread1         float64
 16  relative_spread2         float64
 17  bid_price_diff           float64
 18  ask_price_diff           float64
 19  mid_price_diff           float64
 20  order_imbalance_level1   float64
 21  order_imbala